In [2]:
from labvision import images
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [3]:
im = images.read_img("/media/data/Data/GeneralSystem/MeasureHIPSDimples/P10013/1,96mm_edges.png")

In [4]:
crop_result = images.crop_rectangle(im)

In [5]:
im = images.crop(im, crop_result.bbox)

In [7]:
images.save(im, "/home/ppxjd3/Thesis/Thesis/Chapters/Different Spacings/Figures/ball_contours.png")

In [6]:
images.display(im)

[]

In [7]:
im = images.bgr_to_gray(im)

In [6]:
contours, hier = cv2.findContours(im, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

error: OpenCV(4.5.3) /tmp/pip-req-build-3umofm98/opencv/modules/imgproc/src/contours.cpp:195: error: (-210:Unsupported format or combination of formats) [Start]FindContours supports only CV_8UC1 images when mode != CV_RETR_FLOODFILL otherwise supports CV_32SC1 images only in function 'cvStartFindContours_Impl'


In [ ]:
im_draw = images.draw_contours(images.gray_to_bgr(im), contours_parents)
images.display(im_draw)

In [12]:
contours_parents = [c for c, h in zip(contours, hier.squeeze()) if h[0] != -1]

In [25]:
areas = [cv2.contourArea(c) for c in contours_parents]
circles = [cv2.minEnclosingCircle(c) for c in contours_parents]

In [26]:
contours_parents_correct_area = [c for c, a in zip(contours_parents, areas) if 500 < a < 2500]

In [27]:
circles = [cv2.minEnclosingCircle(c) for c in contours_parents_correct_area]

In [28]:
centers, radius = zip(*circles)

In [29]:
x, y = zip(*centers)

In [30]:
plt.plot(x, y, 'x')
for c in contours_parents_correct_area:
    plt.plot(c[:, :, 0], c[:, :, 1])

In [31]:
from scipy import spatial

In [32]:
tree = spatial.KDTree(centers)

In [33]:
dists, indices = tree.query(centers, 7)

In [37]:
dists_flat = np.ndarray.flatten(dists)
dists_flat = dists_flat[np.nonzero((65 < dists_flat)* (dists_flat <95))]

In [38]:
plt.hist(dists_flat, bins=50)
plt.axvline(np.mean(dists_flat))
plt.axvline(np.mean(dists_flat)-np.std(dists_flat)/np.sqrt(len(dists_flat)))
plt.axvline(np.mean(dists_flat)+np.std(dists_flat)/np.sqrt(len(dists_flat)))

In [38]:
plt.imshow(im)

## Try fitting a gaussian properly

In [39]:

from scipy.stats import norm
import matplotlib.mlab as mlab

In [51]:
mu, sigma = norm.fit(dists_flat)
n, bins, patches = plt.hist(dists_flat, bins=50, density=True)
y = norm.pdf(bins, mu, sigma)
l, = plt.plot(bins, y, 'r--')
plt.title('1.96mm distances')
plt.xlabel('Distance (pixels)')
plt.axvline(mu, c='m')

In [42]:
mu, sigma

(77.34098070539736, 2.0519386503635695)

In [92]:
dists_flat = np.ndarray.flatten(dists)
dists_flat = dists_flat[np.nonzero((65 < dists_flat)* (dists_flat <90))]
mu, sigma = norm.fit(dists_flat)
mu

76.86762626951801

In [52]:
def measure_separation_of_contours(im, min_area=500, max_area=2500, min_dist=65, max_dist=95, title=None):
    contours, hier = cv2.findContours(im, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = [c for c, h in zip(contours, hier.squeeze()) if h[0] != -1]
    areas = [cv2.contourArea(c) for c in contours]
    contours = [c for c, a in zip(contours, areas) if min_area < a < max_area]
    circles = [cv2.minEnclosingCircle(c) for c in contours]
    centers, radius = zip(*circles)
    tree = spatial.KDTree(centers)
    dists, indices = tree.query(centers, 7)
    dists = np.ndarray.flatten(dists[:, 1:])
    dists = dists[np.nonzero((dists>min_dist)*(dists<max_dist))]
    mu, sigma = norm.fit(dists)
    n, bins, patches = plt.hist(dists, bins=50, density=True)
    y = norm.pdf(bins, mu, sigma)
    l, = plt.plot(bins, y, 'r--')
    plt.title(title)
    plt.xlabel('Distance (pixels)')
    plt.axvline(mu, c='m')
    return mu

In [56]:
from scipy import optimize

In [129]:
def measure_separation_of_contours(im, min_area=500, max_area=2500, min_dist=65, max_dist=95, title=None, n_bins=100):
    contours, hier = cv2.findContours(im, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = [c for c, h in zip(contours, hier.squeeze()) if h[0] != -1]
    areas = [cv2.contourArea(c) for c in contours]
    contours = [c for c, a in zip(contours, areas) if min_area < a < max_area]
    circles = [cv2.minEnclosingCircle(c) for c in contours]
    centers, radius = zip(*circles)
    tree = spatial.KDTree(centers)
    dists, indices = tree.query(centers, 7)
    dists = np.ndarray.flatten(dists[:, 1:])
    dists = dists[np.nonzero((dists>min_dist)*(dists<max_dist))]
    n, bins, patches = plt.hist(dists, bins=n_bins, density=True)
    bin_centers = (0.5*(bins[1:]+bins[:-1]))
    mu, sigma = norm.fit(dists)
    pars, cov = optimize.curve_fit(lambda x, mu, sig : norm.pdf(x, loc=mu, scale=sig), bin_centers, n, p0=[mu, sigma])
    # y = norm.pdf(bins, mu, sigma)
    # l, = plt.plot(bins, y, 'r--')
    plt.plot(bin_centers, norm.pdf(bin_centers, *pars), 'k--')
    plt.title(title)
    plt.xlabel('Distance (pixels)')
    plt.axvline(mu, c='m')
    return mu, pars[0], np.sqrt(cov[0, 0]), sigma, sigma/np.sqrt(len(dists))

In [132]:
im_189 = images.read_img("/media/data/Data/GeneralSystem/MeasureHIPSDimples/P10013/1,89mm_edges.png")
im_189 = images.bgr_to_gray(im_189)
mu_189, mu_189_fit, mu_189_err, sigma_189, sigma_189_div_rtN = measure_separation_of_contours(im_189, n_bins=300, title='1.89mm')
mu_189, mu_189_fit, mu_189_err, sigma_189, sigma_189_div_rtN

(74.5893598528276,
 74.53584817832702,
 0.019954525247847743,
 2.4310327470855677,
 0.017324348956062666)

In [98]:
im_195 = images.read_img("/media/data/Data/GeneralSystem/MeasureHIPSDimples/P10013/1,95mm_edges.png")
im_195 = images.bgr_to_gray(im_195)
mu_195, mu_195_fit, mu_195_err = measure_separation_of_contours(im_195, n_bins=300, title='1.95mm')
mu_195, mu_195_fit, mu_195_err

(76.87030746919108, 76.86573198046307, 0.019239826980432237)

In [96]:
bin_numbers = [25, 50, 100, 125, 150, 175, 200, 250, 300]
_, means, errors = zip(*[measure_separation_of_contours(im_196, n_bins=b) for b in bin_numbers])
plt.figure()
plt.errorbar(bin_numbers, means, errors)
plt.xlabel('number_of_bins')
plt.ylabel('Gaussian center')
plt.title('Does number of bins affect gaussian center')


Text(0.5, 1.0, 'Does number of bins affect gaussian center')

In [99]:
im_196 = images.read_img("/media/data/Data/GeneralSystem/MeasureHIPSDimples/P10013/1,96mm_edges.png")
im_196 = images.bgr_to_gray(im_196)
mu_196, mu_196_fit, mu_196_err = measure_separation_of_contours(im_196, title='1.96mm', n_bins=300)
mu_196, mu_196_fit, mu_196_err

(77.34098070539736, 77.27669388679904, 0.01532006394913588)

### Measure size of holes

In [121]:
def measure_size_of_holes(im, min_area=500, max_area=2500, min_rad=1, max_rad=30, n_bins=100, title=''):
    contours, hier = cv2.findContours(im, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = [c for c, h in zip(contours, hier.squeeze()) if h[0] != -1]
    areas = [cv2.contourArea(c) for c in contours]
    contours = [c for c, a in zip(contours, areas) if min_area < a < max_area]
    circles = [cv2.minEnclosingCircle(c) for c in contours]
    centers, radius = zip(*circles)
    radius = np.array(radius)
    radius = radius[np.nonzero((radius>min_rad)*(radius<max_rad))]
    n, bins, patches = plt.hist(radius, bins=n_bins, density=True)
    bin_centers = (0.5*(bins[1:]+bins[:-1]))
    mu, sigma = norm.fit(radius)
    pars, cov = optimize.curve_fit(lambda x, mu, sig : norm.pdf(x, loc=mu, scale=sig), bin_centers, n, p0=[mu, sigma])
    plt.plot(bin_centers, norm.pdf(bin_centers, *pars), 'k--')
    plt.title(title)
    plt.xlabel('Radius (pixels)')
    plt.axvline(mu, c='m')
    return mu, sigma

In [122]:
rad_mean_195, rad_mean_195_err = measure_size_of_holes(im_195)

In [123]:
rad_mean_195, rad_mean_195_err

(22.91257324084901, 1.6262670363670457)